In [1]:
# Import dependencies
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, datasets

import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

In [2]:
# Set processing unit
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [3]:
# Set seeds of random variables
torch.manual_seed(777)
if device == 'cuda:0':
    torch.cuda.manual_seed_all(777)

In [4]:
# Model training settings
batch_size = 32
n_epochs = 20
lr = 0.01 # learning rate
momentum = 0.5

In [5]:
# Image normalization
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

In [6]:
# CIFAR10 dataset preparation
train_dataset = datasets.CIFAR10(root='./data/CIFAR10', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data/CIFAR10', train=False, download=True, transform=transform)

# Load CIFAR10 feeder
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [7]:
# Define network architecture
class MultiLayerPerceptron(nn.Module):
    def __init__(self):
        super(MultiLayerPerceptron, self).__init__()
        self.fc1 = nn.Linear(3 * 32 * 32, 512) # (N, 784, 512), N - batch_size
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)

    def forward(self, x): # Forward propagation
        x = x.view(-1, 3 * 32 * 32) # (N, 28, 28) -> (N, 784)
        output = self.fc1(x) # (N, 784) -> (N, 512)
        output = F.sigmoid(output)
        output = self.fc2(output) # (N, 512) -> (512, 256)
        output = F.sigmoid(output)
        output = self.fc3(output) # (N, 256) -> (N, 10)
        output = F.log_softmax(output, dim=1) # 256 logits -> probability distribution (10 classes)
        return output

In [8]:
# Prepare model training
model = MultiLayerPerceptron().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)

In [9]:
# Train the model
def train(model, train_loader, optimizer, log_interval=5):
    model.train()
    for idx, (x, y) in tqdm(enumerate(train_loader)):
        x = x.to(device)
        y = y.to(device)
        output = model(x)
        loss = criterion(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (idx + 1) % log_interval == 0:
            print('Train epoch: [{}/{} ({:.2f}%)]\tTrain Loss: {:.6f}'.format(idx * len(x), len(train_loader.dataset), 100. * idx / len(train_loader), loss.item()))

In [10]:
# Evaluate the model
def evaluate(model, test_loader):
    model.eval()
    test_loss = .0
    correct = 0

    with torch.no_grad():
        for x, y in tqdm(test_loader):
            x = x.to(device)
            y = y.to(device)
            output = model(x)
            test_loss += criterion(output, y).item()
            prediction = output.max(1, keepdim=True)[1]
            correct += prediction.eq(y.view_as(prediction)).sum().item()

    test_loss /= len(test_loader.dataset) / batch_size
    accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, accuracy

In [11]:
for epoch in range(n_epochs):
    train(model, train_loader, optimizer, log_interval=10000)
    test_loss, accuracy = evaluate(model, test_loader)
    print("\n[Epoch: {}], \tTest Loss: {:.4f},\tAccuracy: {:.2f} %\n".format(epoch, test_loss, accuracy))

1563it [00:12, 129.18it/s]
100%|███████████████████████████████████████████████████████████████████████████████| 313/313 [00:02<00:00, 134.64it/s]



[Epoch: 0], 	Test Loss: 1.9469,	Accuracy: 28.99 %



1563it [00:12, 124.36it/s]
100%|███████████████████████████████████████████████████████████████████████████████| 313/313 [00:02<00:00, 135.54it/s]



[Epoch: 1], 	Test Loss: 1.8324,	Accuracy: 34.82 %



1563it [00:12, 121.49it/s]
100%|███████████████████████████████████████████████████████████████████████████████| 313/313 [00:02<00:00, 128.93it/s]


...


[Epoch: 17], 	Test Loss: 1.4886,	Accuracy: 47.87 %



1563it [00:13, 118.38it/s]
100%|███████████████████████████████████████████████████████████████████████████████| 313/313 [00:02<00:00, 130.52it/s]



[Epoch: 18], 	Test Loss: 1.4786,	Accuracy: 48.24 %



1563it [00:12, 124.71it/s]
100%|███████████████████████████████████████████████████████████████████████████████| 313/313 [00:02<00:00, 131.38it/s]


[Epoch: 19], 	Test Loss: 1.4708,	Accuracy: 48.84 %

